# Sentiment Analysis by `Mr. Harshit Dawar!`

* It is a way to identify the sentiments of the persons using the comments/reviews of the users.
* It can be performed using multiple libraries, but obviously the basic logic remains the same as always.
* In this practical, Twitter Dataset of 1.6 Million Tweets is used to perform the sentiment analysis using the review/tweet.